In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [2]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
From (redirected): https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5&confirm=t&uuid=0dfa0c3f-cb5a-455e-96e3-a63ec0d79d6e
To: /content/horse-or-human.zip
100% 150M/150M [00:02<00:00, 58.9MB/s]


In [3]:
import os
import zipfile
local_zip = '/content/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [6]:
train_hourse = os.path.join('./horse-or-human/horses')
train_human = os.path.join('./horse-or-human/humans')

In [7]:
train_horse_image = os.listdir(train_hourse)
print(train_horse_image[:10])
train_human_image = os.listdir(train_human)
print(train_human_image[:10])

['horse01-6.png', 'horse03-7.png', 'horse36-3.png', 'horse49-5.png', 'horse27-7.png', 'horse03-3.png', 'horse09-6.png', 'horse43-4.png', 'horse47-5.png', 'horse01-2.png']
['human13-07.png', 'human05-03.png', 'human17-17.png', 'human11-26.png', 'human14-16.png', 'human12-13.png', 'human10-16.png', 'human06-21.png', 'human05-02.png', 'human02-04.png']


In [9]:
print('Toatal horse images',len(os.listdir(train_hourse)))
print('Toatal human images',len(os.listdir(train_human)))

Toatal horse images 500
Toatal human images 527


In [11]:
%matplotlib inline
import matplotlib.image as mping
nrows = 4
ncols= 4
pic_index= 0

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape= (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 149, 149, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 73, 73, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 35, 35, 64)       

In [23]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data = ImageDataGenerator(rescale = 1 /255)
train_gernator = train_data.flow_from_directory(
    './horse-or-human/',
    target_size = (300,300),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [29]:
history = model.fit(
    train_gernator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1
)


Epoch 1/15
8/8 [==============================] - 19s 804ms/step - loss: 0.7542 - accuracy: 0.5695
Epoch 2/15
8/8 [==============================] - 7s 857ms/step - loss: 1.1416 - accuracy: 0.5539
Epoch 3/15
8/8 [==============================] - 8s 1s/step - loss: 0.6019 - accuracy: 0.6251
Epoch 4/15
8/8 [==============================] - 8s 952ms/step - loss: 1.2920 - accuracy: 0.6407
Epoch 5/15
8/8 [==============================] - 8s 916ms/step - loss: 0.4333 - accuracy: 0.8643
Epoch 6/15
8/8 [==============================] - 7s 853ms/step - loss: 0.4420 - accuracy: 0.8554
Epoch 7/15
8/8 [==============================] - 8s 954ms/step - loss: 0.1786 - accuracy: 0.9333
Epoch 8/15
8/8 [==============================] - 8s 954ms/step - loss: 0.3118 - accuracy: 0.8832
Epoch 9/15
8/8 [==============================] - 9s 1s/step - loss: 0.1619 - accuracy: 0.9365
Epoch 10/15
8/8 [==============================] - 7s 856ms/step - loss: 0.4018 - accuracy: 0.8220
Epoch 11/15
8/8 [=======

In [34]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path,target_size=(300,300))
    x =image.img_to_array(img)
    x = np.expand_dims(x,axis=0)

    images = np.vstack([x])
    classes = model.predict(images,batch_size=10)
    print(classes[0])
    if classes[0] > 0.5:
        print(fn + 'Is A human')
    else:
        print(fn + 'Is A hourse' )

Saving download.jpg to download.jpg
1/1 [==============================] - 0s 19ms/step
[0.]
download.jpgIs A hourse
